# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

## Unity Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name=r"C:\Users\jofan\rl\deep-reinforcement-learning\p3_collab-compet\Tennis_Windows_x86_64\Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

## Multi-agent DDPG

In [ ]:
import multi_ddpg_agent
import model
from matplotlib.pylab import plt

In [ ]:
import importlib
importlib.reload(multi_ddpg_agent)


In [ ]:
# Set parameters
params = {}

params['buffer_size'] = int(1e5)
params['batch_size'] = 256
params['gamma'] = 0.99
params['tau'] = 3e-1
params['lr_actor'] = 1e-4
params['lr_critic'] = 1e-4
params['weight_decay'] = 0
params['action_size'] = 2
params['state_size'] = 24
params['num_steps_per_update'] = 1
config = {'params': params,
          'random_seeds': [6,6],
          'num_agents': 2}



In [ ]:
agent = multi_ddpg_agent.MultiAgent(config)

In [ ]:
import torch

In [ ]:
from collections import deque

def multi_ddpg(agent, env, n_episodes=10000, print_every=20):
    scores_deque = deque(maxlen=print_every)
    scores_window = deque(maxlen=100)
    scores = []
    brain_name = env.brain_names[0]
    min_score_goal = 0.5
    solved = False

    for i_episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations
        agent.reset()
        score = np.zeros(len(env_info.agents))
        
        while True:      
            actions = agent.act(states)
            #print(actions)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done

            agent.step(states, actions, rewards, next_states, dones)
            
            score += rewards
            states = next_states

            if np.any(dones):
                break

        scores_deque.append(np.max(score))
        scores_window.append(np.max(score))
        scores.append(np.max(score))
        if i_episode % print_every == 0:
            print('\rEpisode {}\tScore: {:.2f} \tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque), np.mean(scores_window)))

        if np.mean(scores_window) >= min_score_goal and not solved:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode,
                                                                                         np.mean(scores_window)))
            solved = True
            torch.save(agent.ddpg_agents[0].actor_local.state_dict(), 'agent_0_checkpoint_actor_solved.pth')
            torch.save(agent.ddpg_agents[0].critic_local.state_dict(), 'agent_0_checkpoint_critic_solved.pth')
            torch.save(agent.ddpg_agents[1].actor_local.state_dict(), 'agent_1_checkpoint_actor_solved.pth')
            torch.save(agent.ddpg_agents[1].critic_local.state_dict(), 'agent_1_checkpoint_critic_solved.pth')
            
        if np.mean(scores_window) >= (min_score_goal*2):
            break

    torch.save(agent.ddpg_agents[0].actor_local.state_dict(), 'agent_0_checkpoint_actor.pth')
    torch.save(agent.ddpg_agents[0].critic_local.state_dict(), 'agent_0_checkpoint_critic.pth')
    torch.save(agent.ddpg_agents[1].actor_local.state_dict(), 'agent_1_checkpoint_actor.pth')
    torch.save(agent.ddpg_agents[1].critic_local.state_dict(), 'agent_1_checkpoint_critic.pth')


    return scores

In [ ]:
scores = multi_ddpg(agent, env)

In [ ]:
scores_ma = scores_df.rolling(window=100, min_periods=1).mean().values
fig = plt.figure(figsize=(10,6))
plt.plot(np.arange(len(scores)), scores)
plt.plot(np.arange(len(scores_ma)), scores_ma)
plt.title('Tennis - DDPG')
plt.xlabel('episode')
plt.ylabel('Average Score')
plt.grid()
plt.show()
fig.savefig('result-ddpg.png')